# Send Messages to Teti Kusmiati

This notebook demonstrates how to:
1. Search for Teti in WhatsApp contacts
2. Send her a lovely text message
3. Send her an audio message from a local file


In [ ]:
# Import required libraries
import requests
import json
from datetime import datetime
from typing import Dict, List, Any
import mcp
from mcp.client.stdio import stdio_client


In [ ]:
# Configuration
WHATSAPP_MCP_SSE_URL = "https://whatsapp-mcp-route-whatsapp-mcp.apps.rosa.akram.a1ey.p3.openshiftapps.com/sse"
AUDIO_FILE_PATH = "/Users/akram/Downloads/message.mp3"

print(f"🔧 Configuration:")
print(f"   WhatsApp MCP SSE URL: {WHATSAPP_MCP_SSE_URL}")
print(f"   Audio File: {AUDIO_FILE_PATH}")
print(f"   Using MCP protocol for tool discovery and execution")


In [ ]:
async def connect_to_mcp_server():
    """Connect to the WhatsApp MCP server using SSE transport."""
    print("🔌 Connecting to WhatsApp MCP server...")
    
    try:
        # Create MCP client with SSE transport
        client = mcp.client.sse.sse_client(WHATSAPP_MCP_SSE_URL)
        
        # Initialize the connection
        await client.initialize()
        
        print("✅ Connected to WhatsApp MCP server!")
        return client
        
    except Exception as e:
        print(f"❌ Failed to connect to MCP server: {e}")
        return None

# Connect to MCP server
mcp_client = await connect_to_mcp_server()


In [ ]:
async def discover_tools():
    """Discover available WhatsApp MCP tools."""
    print("🔍 Discovering available WhatsApp MCP tools...")
    
    if not mcp_client:
        print("❌ No MCP client available")
        return []
    
    try:
        # List available tools
        tools = await mcp_client.list_tools()
        
        print(f"✅ Found {len(tools.tools)} WhatsApp MCP tools:")
        for tool in tools.tools:
            print(f"   - {tool.name}: {tool.description}")
        
        return tools.tools
        
    except Exception as e:
        print(f"❌ Failed to discover tools: {e}")
        return []

# Discover tools
available_tools = await discover_tools()


In [ ]:
async def search_contacts_mcp(query: str):
    """Search WhatsApp contacts using MCP tool."""
    print(f"🔍 Searching for contacts matching: '{query}'")
    
    if not mcp_client:
        print("❌ No MCP client available")
        return []
    
    try:
        # Call the search_contacts tool via MCP
        result = await mcp_client.call_tool("search_contacts", {"query": query})
        
        if result.content:
            contacts = result.content[0].text
            contacts_data = json.loads(contacts)
            print(f"✅ Found {len(contacts_data)} contacts")
            return contacts_data
        else:
            print("❌ No contacts returned")
            return []
            
    except Exception as e:
        print(f"❌ Search error: {e}")
        return []

# Search for Teti using MCP
contacts = await search_contacts_mcp("Teti")

if contacts:
    print("\n📋 Found contacts:")
    for i, contact in enumerate(contacts, 1):
        print(f"   {i}. {contact.get('name', 'Unknown')} - {contact.get('phone_number', 'No phone')}")
else:
    print("\n❌ No contacts found for 'Teti'")


In [ ]:
async def send_text_message_mcp(recipient: str, message: str):
    """Send a text message using MCP tool."""
    print(f"📤 Sending text message to: {recipient}")
    print(f"💬 Message: {message}")
    
    if not mcp_client:
        print("❌ No MCP client available")
        return False
    
    try:
        # Call the send_message tool via MCP
        result = await mcp_client.call_tool("send_message", {
            "recipient": recipient,
            "message": message
        })
        
        if result.content:
            response = result.content[0].text
            print(f"✅ Text message sent successfully!")
            print(f"   Result: {response}")
            return True
        else:
            print("❌ No response from send_message tool")
            return False
            
    except Exception as e:
        print(f"❌ Text message error: {e}")
        return False

# Send lovely text message to Teti using MCP
if contacts:
    # Use the first contact found (assuming it's Teti)
    teti_contact = contacts[0]
    teti_phone = teti_contact.get('phone_number')
    
    if teti_phone:
        lovely_message = "Hello Teti! 👋 Hope you're having a wonderful day! This message was sent using WhatsApp MCP integration. Sending you lots of positive vibes! 🌟✨"
        
        text_success = await send_text_message_mcp(teti_phone, lovely_message)
    else:
        print("❌ No phone number found for Teti")
        text_success = False
else:
    print("❌ No contacts found, cannot send message")
    text_success = False


In [ ]:
async def send_audio_message_mcp(recipient: str, audio_file_path: str):
    """Send an audio message using MCP tool."""
    print(f"🎵 Sending audio message to: {recipient}")
    print(f"📁 Audio file: {audio_file_path}")
    
    # Check if file exists
    import os
    if not os.path.exists(audio_file_path):
        print(f"❌ Audio file not found: {audio_file_path}")
        return False
    
    if not mcp_client:
        print("❌ No MCP client available")
        return False
    
    try:
        # Call the send_audio_message tool via MCP
        result = await mcp_client.call_tool("send_audio_message", {
            "recipient": recipient,
            "media_path": audio_file_path
        })
        
        if result.content:
            response = result.content[0].text
            print(f"✅ Audio message sent successfully!")
            print(f"   Result: {response}")
            return True
        else:
            print("❌ No response from send_audio_message tool")
            return False
            
    except Exception as e:
        print(f"❌ Audio message error: {e}")
        return False

# Send audio message to Teti using MCP
if contacts and 'teti_phone' in locals():
    audio_success = await send_audio_message_mcp(teti_phone, AUDIO_FILE_PATH)
else:
    print("❌ No contacts found or phone number not available, cannot send audio")
    audio_success = False


In [ ]:
# Summary of results
print("\n" + "="*50)
print("📊 SUMMARY")
print("="*50)

print(f"🔌 MCP Connection: {'✅ Success' if mcp_client else '❌ Failed'}")
print(f"🔧 Tools Discovered: {'✅ Found ' + str(len(available_tools)) + ' tools' if available_tools else '❌ No tools found'}")
print(f"🔍 Contact Search: {'✅ Found ' + str(len(contacts)) + ' contacts' if contacts else '❌ No contacts found'}")

if contacts:
    teti_contact = contacts[0]
    print(f"👤 Teti Contact: {teti_contact.get('name', 'Unknown')} - {teti_contact.get('phone_number', 'No phone')}")

print(f"💬 Text Message: {'✅ Sent' if 'text_success' in locals() and text_success else '❌ Failed'}")
print(f"🎵 Audio Message: {'✅ Sent' if 'audio_success' in locals() and audio_success else '❌ Failed'}")

print("\n🎉 Mission completed! Check Teti's WhatsApp for the messages! 💌")
print("🔧 All operations used MCP protocol for tool discovery and execution!")
